### Cài đặt thư viện

### Import thư viện

In [4]:
import mmcv

from mmdet.apis import init_detector, inference_detector
from mmengine import Config
from mmengine.runner import Runner, set_random_seed
from mmengine.visualization import Visualizer

from pathlib import Path
from PIL import Image

from pycocotools import _mask as coco_mask
from pycocotools.coco import COCO

from skimage import io
from skimage.morphology import binary_dilation

from tqdm import tqdm

import base64
import cv2

import glob
import json

import random
import matplotlib.pyplot as plt

import numpy as np
import os
import pandas as pd

import torch

import torchvision.transforms as T
import typing as t

import zlib
from collections import defaultdict

from sklearn.model_selection import train_test_split
from copy import deepcopy

In [11]:
%cd mmdetection

/kaggle/working/mmdetection


In [13]:
!mim download mmdet --config centripetalnet_hourglass104_16xb6-crop511-210e-mstest_coco --dest .

/opt/conda/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (None)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn(
processing centripetalnet_hourglass104_16xb6-crop511-210e-mstest_coco...
downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.6/785.6 MiB 36.5 MB/s eta 0:00:0000:0100:01
Successfully downloaded centripetalnet_hourglass104_mstest_16x6_210e_coco_20200915_204804-3ccc61e5.pth to /kaggle/working/mmdetection
Successfully dumped centripetalnet_hourglass104_16xb6-crop511-210e-mstest_coco.py to /kaggle/working/mmdetection


### Config model

In [18]:
%%writefile ./configs/centripetalnet/centripetalnet_hourglass104_16xb6-crop511-210e-mstest_sino_nom.py
# Inherit and overwrite part of the config based on this config
_base_ = './centripetalnet_hourglass104_16xb6-crop511-210e-mstest_coco.py'

data_root = '/kaggle/input/mmdetection-test/augmented_data_v1/' # dataset root

train_batch_size_per_gpu = 4
train_num_workers = 4

max_epochs = 20
stage2_num_epochs = 1
base_lr = 0.00008


metainfo = {
    'classes': ('0', ),
    'palette': [
        (220, 20, 60),
    ]
}

train_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='train/images/'),
        ann_file='train.json'))

val_dataloader = dict(
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='valid/images/'),
        ann_file='valid.json'))

test_dataloader = val_dataloader

val_evaluator = dict(ann_file=data_root + 'valid.json')

test_evaluator = val_evaluator

model = dict(bbox_head=dict(num_classes=1), test_cfg=dict(score_thr=0.5, max_per_img=400, corner_topk=400))

# learning rate
# param_scheduler = [
#     dict(
#         type='LinearLR',
#         start_factor=1.0e-5,
#         by_epoch=False,
#         begin=0,
#         end=10),
#     dict(
#         # use cosine lr from 10 to 20 epoch
#         type='CosineAnnealingLR',
#         eta_min=base_lr * 0.05,
#         begin=max_epochs // 2,
#         end=max_epochs,
#         T_max=max_epochs // 2,
#         by_epoch=True,
#         convert_to_iter_based=True),
# ]

train_pipeline_stage2 = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='RandomResize',
        scale=(640, 640),
        ratio_range=(0.1, 2.0),
        keep_ratio=True),
    dict(type='RandomCrop', crop_size=(640, 640)),
    dict(type='YOLOXHSVRandomAug'),
    dict(type='RandomFlip', prob=0.5),
    dict(type='Pad', size=(640, 640), pad_val=dict(img=(114, 114, 114))),
    dict(type='PackDetInputs')
]

# optimizer
# optim_wrapper = dict(
#     _delete_=True,
#     type='OptimWrapper',
#     optimizer=dict(type='AdamW', lr=base_lr, weight_decay=0.05),
#     paramwise_cfg=dict(
#         norm_decay_mult=0, bias_decay_mult=0, bypass_duplicate=True))

default_hooks = dict(
    checkpoint=dict(
        interval=5,
        max_keep_ckpts=2,  # only keep latest 2 checkpoints
        save_best='auto'
    ),
    logger=dict(type='LoggerHook', interval=20))

custom_hooks = [
    dict(
        type='PipelineSwitchHook',
        switch_epoch=max_epochs - stage2_num_epochs,
        switch_pipeline=train_pipeline_stage2)
]

# load COCO pre-trained weight
load_from = './centripetalnet_hourglass104_mstest_16x6_210e_coco_20200915_204804-3ccc61e5.pth'

train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=max_epochs, val_interval=1)
visualizer = dict(vis_backends=[dict(type='LocalVisBackend'),dict(type='TensorboardVisBackend')])

Overwriting ./configs/centripetalnet/centripetalnet_hourglass104_16xb6-crop511-210e-mstest_sino_nom.py


### Training

In [19]:
!python tools/train.py configs/centripetalnet/centripetalnet_hourglass104_16xb6-crop511-210e-mstest_sino_nom.py

/opt/conda/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (None)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn(
05/22 08:48:00 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 | packaged by conda-forge | (main, Dec 23 2023, 15:36:39) [GCC 12.3.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 230910128
    GPU 0: Tesla P100-PCIE-16GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.1, V12.1.105
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 2.1.2
    PyTorch compiling details: PyTorch built with:
  - GCC 9.4
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
